In [ ]:
!pip install pycuda

In [ ]:
import numpy as np
import pycuda.autoinit
from pycuda import gpuarray
from pycuda.elementwise import ElementwiseKernel
from pycuda.compiler import SourceModule

# Create an initial array to do some work with
data = np.array([1, 2, 3, 4, 5], dtype=np.float32)
data_gpu = gpuarray.to_gpu(data)

# Create an array to hold the output data
# "empty_like" creates an empty array of the same size and shape as another
result_gpu = gpuarray.empty_like(data_gpu)

cuda_module = SourceModule("""
__global__ void scalar_math(float *input, float *result) {
  result[threadIdx.x] = ( 10 * input[threadIdx.x] ) + 1;
}

__global__ void scalar_math2(float *input, float *result, float add) {
  result[threadIdx.x] = ( 10 * input[threadIdx.x] ) + add;
}
""")
scalar_math = cuda_module.get_function("scalar_math")

# Do the calculation on the GPU
scalar_math( data_gpu, result_gpu, block=(5,1,1) )

# Get the result back
result = result_gpu.get()
print(f"Result of scalar_math():  {result}")